In [30]:
import bs4
import jdatetime
import json
import wget
from jdatetime import date
import pandas as pd

In [2]:
base_url='https://trvotes.parliran.ir'

In [24]:
import requests

cookies = {
    '_ga': 'GA1.1.180246938.1693921998',
    '_ga_1X68XESZ48': 'GS1.1.1694898322.8.1.1694898975.0.0.0',
    '__RequestVerificationToken': 'PHmzcFNrFtyFpu7HwJSfbNpA7KO5AEqmZCQr24aQ4XryxRn6cFFkJDDeGcuanJ8kzzYFnOQFliUsHm5k36Yv4g4bwgQ1pNfZMM7evLtDC3E1',
    'AHAS': 'qnwx3zdternqxkesa1w1df5p',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Type': 'multipart/form-data; boundary=----WebKitFormBoundaryL34cAhyYd4BIn2Qi',
    # 'Cookie': '_ga=GA1.1.180246938.1693921998; _ga_1X68XESZ48=GS1.1.1694898322.8.1.1694898975.0.0.0; __RequestVerificationToken=PHmzcFNrFtyFpu7HwJSfbNpA7KO5AEqmZCQr24aQ4XryxRn6cFFkJDDeGcuanJ8kzzYFnOQFliUsHm5k36Yv4g4bwgQ1pNfZMM7evLtDC3E1; AHAS=qnwx3zdternqxkesa1w1df5p',
    'Origin': 'https://trvotes.parliran.ir',
    'Referer': 'https://trvotes.parliran.ir/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.81',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Microsoft Edge";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}
today=date.today().strftime('%Y/%m/%d')
data = f'------WebKitFormBoundaryL34cAhyYd4BIn2Qi\r\nContent-Disposition: form-data; name="__RequestVerificationToken"\r\n\r\nF3xgpM9aQ2c2EMDMrmaQz7nzHqqbo_wyMdcwauvgnOICPqwnaIuVTa_FsxK7MGjUWJx7Ef0DTSwbGZXv1Cc2aMcY02T0H-2ayC5ZR3HW62w1\r\n------WebKitFormBoundaryL34cAhyYd4BIn2Qi\r\nContent-Disposition: form-data; name="StartTime"\r\n\r\n1401/06/26\r\n------WebKitFormBoundaryL34cAhyYd4BIn2Qi\r\nContent-Disposition: form-data; name="EndTime"\r\n\r\n{today}\r\n------WebKitFormBoundaryL34cAhyYd4BIn2Qi\r\nContent-Disposition: form-data; name="myTable_length"\r\n\r\n50\r\n------WebKitFormBoundaryL34cAhyYd4BIn2Qi--\r\n'

response = requests.post('https://trvotes.parliran.ir/', cookies=cookies, headers=headers, data=data ,verify=False)
with open('index.html','wb',) as f:
  f.write( response.content)

c:\Users\muhammadS\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trvotes.parliran.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [25]:
with open("index.html", 'r',encoding='utf8') as f:
    webpage = f.read()

In [26]:
soup = bs4.BeautifulSoup(webpage)

In [27]:
votes=soup.find_all('tr')[1:]

In [28]:
votes_parsed=[]
for vote in votes:
    title,time,url=vote.findAll('td')
    title=' '.join( title.text.split())
    time=time.text.replace('\n','').replace(' ','')
    # time=jdatetime.datetime.strptime(time,'%Y/%m/%d')
    url=base_url+url.find('a')['href']
    votes_parsed.append({'title':title,'time':time,'url':url})

In [29]:
with open('parsed.json','w',encoding='utf8') as f:
    t=json.dumps(votes_parsed,ensure_ascii=False)
    f.write(t)

In [39]:
import tqdm
# votes_parsed=filter(lambda x:  int(vot['time'].replace("/",''))
#   <=14020523,votes_parsed)

for vot in tqdm.tqdm(votes_parsed):
   id=vot['url'].split('/')[-1]
   wget.download(vot['url'],out=f'pages/{id}.html')

100%|██████████| 842/842 [14:08<00:00,  1.01s/it]


In [8]:
members=dict()
for page in os.listdir('pages/'):
    with open(f"pages/{page}", 'r',encoding='utf8') as f:
        webpage = f.read()
        webpage.replace("\\u200c",'')
        print(page)
        soup = bs4.BeautifulSoup(webpage)
        break
        

10.html


In [ ]:
stat_dict={
            '----':'absence',
            'عدم مشارکت':"non-participant",
            'مخالف':'against',
            'موافق':'favor',
            'ممتنع':'abstaining'
        }
def get_day_data(soup,date_:str):
    vote_title=soup.select_one('#page-wrapper > div.row > div.col-lg-12 > div > div.panel-footer').text
    vote_title=' '.join( vote_title.split())
    stats=soup.select('.inner h3')
    favor,against,abstaining ,members_count=map(lambda x: x.text,stats)
    data=soup.find_all('tr')[1:]
    votes=[]
    for row in data:
        row=row.find_all('th')

        img_url=row[0].find('img')['src']
        mem_id=img_url.split('/')[-1].split('.')[0]
        if os.path.exists(f'images/{img_url.split("/")[-1]}')== False:
            wget.download(base_url+img_url,f'images/{img_url.split("/")[-1]}')
        family_city=' '.join(row[2].text.split())
        paran=family_city.index('(')

        

        d={
        'img_url':img_url,
        'id':mem_id,
        'name':' '.join(row[1].text.split()),
        'date':date_,
        'family_city':family_city,
        'family':family_city[:paran],
        'city':family_city[paran:].replace('(','').replace(')',''),
        'stat':stat_dict[ ' '.join(row[4].text.split())]
        }
        if d['id'] not in members:
            members[d['id']]=[
                d['name'],
                d['family'],
                d['city']]
        votes.append(d)
    return [favor,against,abstaining,date_ ,members_count,votes]
get_day_data(soup,'//')

In [10]:
soup.find_all('tr')[1:].__len__()

79